In [8]:

import os, sys, glob

iternum = 0
path_runmodel0 = '/glade/scratch/guoqiang/moasmo_test/run_model'
path_runmodel = f'/glade/scratch/guoqiang/moasmo_test/run_model/iter{iternum}'
os.makedirs(path_runmodel, exist_ok=True)

param_path = '/glade/scratch/guoqiang/moasmo_test/param_sets'

# param_filelist = ['p1.csv', 'p2.csv', 'p3.csv']
param_filelist = glob.glob(f'{param_path}/*iter{iternum}_trial*.csv')
param_filelist.sort()

script_singlerun = '/glade/u/home/guoqiang/CTSM_repos/moasmo_test/run_one_paramset.py'
script_clone = '/glade/u/home/guoqiang/CTSM_repos/CTSM/cime/scripts/create_clone'
path_CTSM_base = '/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100'
path_archive = '/glade/scratch/guoqiang/moasmo_test/ctsm_outputs'
date_start = '1994-10-01'
date_end = '1998-09-30'
ref_streamflow = '/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Lump_calib_split_nest_LMWG/CAMELS_100_OstCalib/refdata/streamflow_data.csv'
add_flow_file = 'nofile'

In [13]:

# create command file
commands_run_model = f'{path_runmodel}/commands_run_iter{iternum}.txt'
with open(commands_run_model, 'w') as f:
    for i in range(len(param_filelist)):
        caseflag = f'iter{iternum}_trial{i}'
        file_parameter_set = f'{param_path}/paramset_iter{iternum}_trial{i}.csv'
        commandi = f"python {script_singlerun} {script_clone} {path_CTSM_base} {file_parameter_set} {path_archive} {caseflag} {date_start} {date_end} {ref_streamflow} {add_flow_file}"
        f.write(f'{commandi}\n')


In [11]:

# create submission file
script_submission = f'{path_runmodel}/submit_iter{iternum}.sh'
cpus = 36 # note that Cheyenne charges for the entire node even this value is smaller than 36

lines = ['#PBS -N moasmo', '#PBS -q regular',
         '#PBS -l walltime=6:00:00', '#PBS -A P08010000',
         f'#PBS -l select=1:ncpus={cpus}',
         '\n',
         'module load conda/latest parallel cdo', 'conda activate npl-2022b',
         '\n',
         'export MPI_DSM_DISTRIBUTE=0',
         '\n'
         f"echo 'Running {commands_run_model}'",
         f'parallel --jobs {cpus} --joblog joblog.txt < {commands_run_model}']

with open(script_submission, 'w') as f:
    for li in lines:
        _ = f.write(li + '\n')

os.system(f'chmod +x {script_submission}')

0